# docker服务搭建（ubuntu）

详见 setup_docker.sh 脚本


# 1. 后端环境搭建

## 1.1 准备虚拟环境

```shell
conda create -n ragflow-video python=3.10
conda activate ragflow-video
```


## 1.2 git克隆项目

将 ragflow 项目clone到当前用户目录下（根据自己机器的情况修改ragflow项目源码的目录）。


```shell
cd ~
# 进入工作目录
cd ～/work-max
# clone项目
git clone https://github.com/infiniflow/ragflow.git
# 进入项目目录
cd ragflow

```


## 1.3 安装uv管理工具

uv:一个极其快速的 Python 包和项目管理器，用 Rust 编写。一款可替代 pip、pip-tools、pipx、poetry、pyenv、twine、virtualenv 等工具的一体化工具。uv和python无关。
RAGFlow这个项目，使用uv来作为项目包的管理工具。

安装：

```shell
curl -LsSf https://astral.sh/uv/install.sh | sh
# 验证是否安装成功：
uv --version
# 安装ragflow项目依赖包
uv sync --python 3.10 --all-extras
```



## 1.4 安装jemalloc 
系统级。jemalloc 是一款高效的内存分配器（malloc 替代库），最初由 FreeBSD 团队开发，后被广泛用于各类高性能应用中。其核心作用是：
优化内存分配效率/提升程序稳定性/支持性能分析。


```shell
# 1. 更新 Homebrew 包索引
brew update

# 2. 安装 jemalloc
brew install jemalloc

# 3. 检查库文件是否存在
ls $(brew --prefix)/lib/libjemalloc.dylib  # Homebrew 安装的情况
# 输出应为：/opt/homebrew/lib/libjemalloc.dylib（或对应路径）
```


## 1.5 修改RAGFlow 项目配置

### 1.5.1 修改 /etc/hosts

注意将这里的127.0.0.1 改成对应docker服务的IP地址。

```shell
grep -qxF '127.0.0.1       es01 infinity mysql minio redis' /etc/hosts || echo '127.0.0.1       es01 infinity mysql minio redis' | sudo tee -a /etc/hosts > /dev/null
```


grep -qxF '47.113.96.6       es01 infinity mysql minio redis' /etc/hosts || echo '47.113.96.6       es01 infinity mysql minio redis' | sudo tee -a /etc/hosts > /dev/null

### 1.5.2 修改 docker/service_conf.yaml.template的端口

修改点
- (1) mysql port: 5455
- (2) es port : 1200



### 1.5.3 修改 docker/entrypoint.sh

在文件 docker/entrypoint.sh 中注释：

```shell
# /usr/sbin/nginx
```


### 1.5.4 复制 docker/service_conf.yaml.template 复制一份到 项目的 conf 目录下

### 1.5.5 修改ragflow目录路径

1. docker/entrypoint.sh

```shell
CONF_DIR="/ragflow/conf"
MCP_SCRIPT_PATH="/ragflow/mcp/server/server.py"
```

2. 


## 1.6 安装依赖

主要下载模型、nltk相关数据。

官方推荐的方式:

```shell
python download_deps.py
```

下载模型：

```shell
conda activate ragflow-video
# 安装huggingface的下载工具
pip install -U "huggingface_hub[cli]"

# 配置代理
export HF_ENDPOINT=https://hf-mirror.com

# 手动执行下载（默认目录: ~/.cache/huggingface/hub）
hf download InfiniFlow/text_concat_xgb_v1.0
# models--InfiniFlow--text_concat_xgb_v1.0
hf download InfiniFlow/deepdoc
# models--InfiniFlow--deepdoc
hf download InfiniFlow/huqie
# models--InfiniFlow--huqie
hf download BAAI/bge-large-zh-v1.5
# models--BAAI--bge-large-zh-v1.5
hf download maidalun1020/bce-embedding-base_v1
# models--maidalun1020--bce-embedding-base_v1
```


下载nltk数据：

https://github.com/nltk/nltk_data

1. 下载zip包，解压
2. 将 解压出来的 packages 文件夹 copy 到 ragflow 的 .venv 目录下，并将 packages 改为 nltk_data


## 1.7 启动后台服务

```shell
# 进入项目根目录
cd ~/work-max/ragflow
# 激活虚拟环境
source .venv/bin/activate
export PYTHONPATH=$(pwd)
# 启动
./docker/entrypoint.sh

```


# 1.8 停止后台服务

官方推荐的方式（不生效）

```shell
pkill docker/entrypoint.sh
```

简单粗暴的方式
```shell
ps aux | grep ragflow
# 8552
ps aux | grep task_executor.py
# 8553
kill -9 8553 8552
```


# 2. 前端环境搭建

nodejs需要18.20.4之后的版本

## 2.1 安装依赖+启动

```shell
# 安装
brew install node@20
# 如果需要将其设置为系统默认 Node.js
brew link node@20 --force
# 验证
node --version

# 安装项目依赖包
npm i

# 启动项目
npm run dev

```


# 3. 报错问题记录

## 3.1 执行 uv sync --python 3.10 --all-extras 报错
没有 icu-conifg/pkg-config

FileNotFoundError: [Errno 2] No such file or directory: 'icu-config'
...
FileNotFoundError: [Errno 2] No such file or directory: 'pkg-config'

解决方案：

```shell
# 1. 确保 icu4c 安装
brew update
brew install icu4c pkg-config

# 2. 设置环境变量（关键）
export ICU_HOME="/opt/homebrew/opt/icu4c"
export PKG_CONFIG_PATH="/opt/homebrew/opt/icu4c/lib/pkgconfig:$PKG_CONFIG_PATH"
export CFLAGS="-I$ICU_HOME/include $CFLAGS"
export LDFLAGS="-L$ICU_HOME/lib $LDFLAGS"
export CPPFLAGS="-I$ICU_HOME/include $CPPFLAGS"
export LIBRARY_PATH="$ICU_HOME/lib:$LIBRARY_PATH"

# 3. 创建 pkg-config 链接（如果缺失）
ln -sf /opt/homebrew/opt/icu4c/lib/pkgconfig/icu-*.pc /opt/homebrew/lib/pkgconfig/ 2>/dev/null || true

# 4. 验证 pkg-config
pkg-config --cflags icu-uc
pkg-config --libs icu-uc

# 5. 清理并重试安装
uv pip uninstall pyicu -y
uv pip install --no-cache-dir --force-reinstall --no-binary pyicu pyicu==2.15.3

# 6. 再运行 sync
uv sync --python 3.10 --all-extras
```
